In [1]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain.schema.runnable import RunnableLambda

model = OllamaLLM(model="llama3.2:1b")
# model = OllamaLLM(model="gpt-oss:20b")

In [2]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞),我會自已手動增加"),
    ("human","請回覆這封客戶郵件：\n\n{email_content}")
])

In [3]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("中華民國,西元%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    return formatted_reply

format_email = RunnableLambda(format_email_reply)

In [4]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [5]:
chain = (prompt_template 
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
)

In [6]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。


王小明"""

In [7]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

系統：郵件品質評估 tools 中，王小明的郵件呈現如下：

專業度 (1-10分): 6/10
友善度 (1-10分): 8/10
完整度 (1-10分): 9/10

分析：

優點:

* 輕盈：郵件內的語言表達顯示王小明對公司的 products 關心，這是很好的品質。
* 透明度：郵件提供了相關政策和程序的資訊，這使得王小明更容易了解公司的行為。

缺點：

* 輕盈不充分：郵件中沒有直接表達謝意，也未提及對公司的感謝，可能會讓人們在看完郵件後感到無序。
* 內容性：
	+ 對 company 的政策和程序沒有關心，這可能會給王小明或其他Customer感到不信任。
	+ 只提到 product ID、購買時間點以及詳情，但未直接表達謝意和感謝。

改進建議：

1. 燦光王小明：用個人的字體和標記來強調這封郵件，例如使用「感謝你的郵件」或是「對您公司的 products 關心」等文字。
2. 提及謝意：直接表達謝意，例如「感謝您的專業幫助，我非常感謝您對我公司的 products 關心」，這可以顯示王小明對 company 的感激情況。
3. 增加內容性：
	+ 將相關政策和程序中提到的 details（如 product ID、購買時間點以及詳情）列出來，讓王小明或其他Customer更容易了解。
	+ 提到任何需要幫助的細節（例如提供指南和資訊），這可以顯示 company 的支持水平。

綜合上述分析，這封郵件呈現如下：

* 將公司的 products 關心表達得很好，但缺點較大。
* 將謝意表達得太少，可能會影響王小明對 company 的感激情況。
* 內容性部分有所改進，但仍然可以增加。

最終評估：這封郵件的專業度和完整度是 6/10、9/10。王小明的謝意表達得很好，但未直接表達對 company 的感激情況，需要加強謝意的表達，以提高郵件的整體品質和信任程度。
